<a href="https://colab.research.google.com/github/naveedkhalid091/Learn_Agentic_AI/blob/main/step02_generative_ai_for_beginners/02(c)_Advance_RAG_Picture%26audio_recognition_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Face & Audio Detection with Embeddings:**

With introduction of this face detection technology, AI can now setup **security system** for not only Humans but also for the Animals as well.

You can introduce Animal's Passport with the help of this technology becasue every animal has unique attributes from thier nose.   

- You can install the **`facenet`** for face dedection.
- You can also install the **`YAMNet`**  for voice/sound recognition.

With the Introduction of above two technologies you can now ask LLM to make **written notes**  for you from a single video (Zoom Lecture) with the help of `facenet` & `YAMNet`. **`facenet`** will recognise the face from the video and YAMnet will regognise the voice of that person and LLM will produce written notes based on the wordings of your ZOOM class.





In [ ]:
!pip install -U -q facenet-pytorch

In [ ]:
!pip install -U -q pillow

In [ ]:
import torch # this is by defalut installed in colab files

import torch.nn as nn  ## nn means neural network
import torchvision.transforms as transforms
from PIL import Image


In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1 # this is architecture

model = InceptionResnetV1(pretrained='vggface2').eval()
model  # The complete architechture of this model is here now

**Note: Now if we will give any Human or animal piture to above model then that model will embedd the important features of the relevant face.**

In [ ]:
# preprocessing function to transform the Image into a tensor
def preprocess_image(image_path):
  image=Image.open (image_path).convert('RGB')
  preprocess = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  return preprocess(image).unsqueeze(0)

  # Function to create image embeddings

  def create_image_embeddings(image_paths):
    try:
      input_tensor = preprocess_image(image_path)
      with torch.no_grad():
        embeddings = model(input_tensor) # embedding important line
      return embeddings.squeeze().numpy()
    except Exception as e:
      print("Error:",e)
      return None

In [ ]:
!mkdir images # Create images folder in directory

In [ ]:
# importing pictures from url

import requests
import os

def save_image_from_url(image_url, image_name):
  """
  Download an image from URL and saves it to the "image" folder.

  Agrs:
  image_url: The url of thre image to download
  image_name: The name of file to save the image as:
  """
  try:
    if not os.path.exists("images"):
      os.makedirs("images")

    image_path=os.path.join("images",image_name)
    response=requests.get(image_url, stream=True)
    response.raise_for_status() # Raise exception for bad status codes:

    with open(image_path, 'wb') as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

    print(f"Image saved to: {image_path}")
  except requests.exceptions.RequestException as e:
    print(f"Error downloading image: {e}")
  except Exception as e:
    print(f"Error saving image: {e}")

In [ ]:
save_image_from_url("https://www.facebook.com/photo/?fbid=6506220269475611&set=a.187728274658207", "Khalid.jpg")

Image saved to: images/Khalid.jpg


In [ ]:
# Example usage

image_path2="/content/images/DSC06817.JPG"
abu=create_image_embeddings(image_path2)

NameError: name 'create_image_embeddings' is not defined